In [1]:
import advent

def parse_line(line):
    words = line.split(' ')
    return tuple([int(words[ix]) for ix in [6, 12, 18, 21, 27, 30]])

lines = advent.get_lines(19, map_fn=parse_line)

In [2]:
tadd3 = lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2])
tadd4 = lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2], a[3] + b[3])
tmul = lambda c, a: (c * a[0], c*a[1], c*a[2], c*a[3])

def build_and_mine(blueprint, resources, miners, current_miners):
    # Checks if a suggested number of miners is possible to build with the given resources
    # and if so, return tuple with resources left after building

    # First, check if we already have enough miners, and if so, return false
    if miners[0] and current_miners[0] >= max(blueprint[0], blueprint[1], blueprint[2], blueprint[4]): return False, False
    if miners[1] and current_miners[1] >= blueprint[3]: False, False
    if miners[2] and current_miners[2] >= blueprint[5]: return False, False

    ore_needed = (miners[0] * blueprint[0]) + (miners[1] * blueprint[1]) + (miners[2] * blueprint[2]) + (miners[3] * blueprint[4])
    clay_needed = (miners[2] * blueprint[3])
    obsidian_needed = (miners[3] * blueprint[5])
    if ore_needed > resources[0] or clay_needed > resources[1] or obsidian_needed > resources[2]:
        return False, False
    new_miners = tadd4(miners, current_miners)
    new_resources = tadd4(resources, tmul(-1, (ore_needed, clay_needed, obsidian_needed, 0)))
    new_resources = tadd4(new_resources, current_miners)
    return new_resources, new_miners

In [ ]:
# Attempt 2: choose the next robot to build, from the list of 4. Then build that next robot ASAP, before generating another robot
# hopefully, we will build maximum like 6 robots, so this will generate 4^6 = 4096 paths, which seems managable
# and many of those paths will be cut off early, such as any path that starts with 'geode' (because you don't have obsidian yet)
# unfortunately in practice we build way more robots than just 6

def amax(*args):
    return max(args, key=lambda x: x[0])

def maximum_geodes_2(blueprint, resources=(0,0,0,0), miners=(1,0,0,0), minute=1, next_build=None, max_minute=24):
    # max_build = 0: ore, 1: clay, 2: obsidian, 3: geode
    if minute == max_minute: return miners[3] + resources[3], []
    if next_build == 2 and miners[1] == 0: return 0, []
    if next_build == 3 and miners[2] == 0: return 0, []

    if next_build is None:
        r1 = maximum_geodes_2(blueprint, resources, miners, minute, 1, max_minute)
        r0 = maximum_geodes_2(blueprint, resources, miners, minute, 0, max_minute)
        if r0[0] > r1[0]: return r0
        return r1


    to_build_tmp = (int(next_build==0), int(next_build==1), int(next_build==2), int(next_build==3))
    new_resources, new_miners = build_and_mine(blueprint, resources, to_build_tmp, miners)
    #print(resources, miners, minute, resources_left, to_build)
    if new_resources:
        # This restriction works for part 1, not for part 2
        # next_build_options = {0: [0, 1], 1:[0, 1, 2], 2: [1, 2, 3], 3: [2, 3]}[next_build]
        next_build_options = [0, 1, 2, 3]
        res = max([maximum_geodes_2(blueprint, new_resources, new_miners, minute+1, i, max_minute) for i in next_build_options], key=lambda x: x[0])
        return res[0], [(new_miners, new_resources, minute+1)] + res[1]
    else:
        # pass the tim
        new_resources = tadd4(resources, miners)
        max_geodes, subresult = maximum_geodes_2(blueprint, new_resources, miners, minute + 1, next_build, max_minute)
        return max_geodes, [(miners, new_resources, minute+1)] + subresult

blueprint1 = (4, 2, 3, 14, 2, 7)
blueprint2 = (2, 3, 3, 8, 3, 12)
m, p = maximum_geodes_2(blueprint1, next_build=None, max_minute=24)


In [ ]:
from tqdm.notebook import tqdm
result = 0

for i, line in tqdm(list(enumerate(lines))):
    result += maximum_geodes_2(line, next_build=None, max_minute=24)[0] * (i+1)

print(result)

In [ ]:
maximum_geodes_2(lines[0], next_build=None, max_minute=32)

In [ ]:
maximum_geodes_2(lines[1], next_build=None, max_minute=32)
# This took like 35 minutes...

In [ ]:
maximum_geodes_2(lines[2], next_build=None, max_minute=32)

In [14]:
# manually copy from previous cells, would of course be easy to do in code
# but it takes so long it's nice to do it piecewise
11*49*25

13475